In [1]:
# !pip install transformers[torch]
# !pip install pandas
# !pip install datasets 

In [1]:
#imports
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BartForConditionalGeneration, BartTokenizer
from transformers import Trainer, TrainingArguments
import pandas as pd
from tqdm.notebook import tqdm

c:\Users\theav\miniconda3\envs\modelfinetune\Lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
c:\Users\theav\miniconda3\envs\modelfinetune\Lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
c:\Users\theav\miniconda3\envs\modelfinetune\Lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [3]:
from datasets import load_dataset

dataset = load_dataset("Samsung/samsum", "samsum", trust_remote_code=True)

corpus.7z:   0%|          | 0.00/2.94M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

In [7]:
dataset['train']

Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 14732
})

In [8]:
from elife_dataset.preprocessing import clean_dataset
dataset = clean_dataset(dataset)

ModuleNotFoundError: No module named 'elife_dataset'

In [12]:
class SAMSumDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        dialogue = item['dialogue']
        summary = item['summary']
        inputs = self.tokenizer(dialogue, return_tensors='pt', max_length=512, truncation=True)
        targets = self.tokenizer(summary, return_tensors='pt', max_length=128, truncation=True)
        return {
            'input_ids': inputs.input_ids.flatten(),
            'attention_mask': inputs.attention_mask.flatten(),
            'labels': targets.input_ids.flatten()
        }

In [10]:
model_name = "facebook/bart-base"
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = BartTokenizer.from_pretrained(model_name)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

c:\Users\theav\miniconda3\envs\modelfinetune\Lib\site-packages\huggingface_hub\file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=

In [13]:
#training_dataset = DailyMailDataset(validation_data, tokenizer)
train = SAMSumDataset(dataset['train'], tokenizer)

In [14]:
#validation_dataset = DailyMailDataset(train_data, tokenizer)
validation = SAMSumDataset(dataset['validation'], tokenizer)

In [16]:
from transformers import DataCollatorForSeq2Seq

# Create a data collator specifically for sequence-to-sequence models
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True,  # This ensures dynamic padding within each batch
    label_pad_token_id=-100  # Ensures labels are padded with -100 for loss calculation
)

training_args = TrainingArguments(
    output_dir='training',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    logging_steps=10,
    fp16=True,
    save_strategy="epoch",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=validation,
    data_collator=data_collator
)


c:\Users\theav\miniconda3\envs\modelfinetune\Lib\site-packages\accelerate\accelerator.py:457: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(
c:\Users\theav\miniconda3\envs\modelfinetune\Lib\site-packages\accelerate\accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [17]:
trainer.train()

  0%|          | 0/5526 [00:00<?, ?it/s]

{'loss': 4.3596, 'learning_rate': 7.000000000000001e-07, 'epoch': 0.01}
{'loss': 4.1931, 'learning_rate': 1.7000000000000002e-06, 'epoch': 0.01}
{'loss': 3.9124, 'learning_rate': 2.7e-06, 'epoch': 0.02}
{'loss': 3.3269, 'learning_rate': 3.7e-06, 'epoch': 0.02}
{'loss': 2.6702, 'learning_rate': 4.7e-06, 'epoch': 0.03}
{'loss': 2.3419, 'learning_rate': 5.7000000000000005e-06, 'epoch': 0.03}
{'loss': 2.4545, 'learning_rate': 6.700000000000001e-06, 'epoch': 0.04}
{'loss': 2.3389, 'learning_rate': 7.6e-06, 'epoch': 0.04}
{'loss': 2.419, 'learning_rate': 8.599999999999999e-06, 'epoch': 0.05}
{'loss': 2.2155, 'learning_rate': 9.600000000000001e-06, 'epoch': 0.05}
{'loss': 2.2253, 'learning_rate': 1.06e-05, 'epoch': 0.06}
{'loss': 2.2492, 'learning_rate': 1.16e-05, 'epoch': 0.07}
{'loss': 2.1021, 'learning_rate': 1.2600000000000001e-05, 'epoch': 0.07}
{'loss': 2.1877, 'learning_rate': 1.3600000000000002e-05, 'epoch': 0.08}
{'loss': 2.1849, 'learning_rate': 1.4599999999999999e-05, 'epoch': 0.08

  0%|          | 0/103 [00:00<?, ?it/s]

{'eval_loss': 1.5865745544433594, 'eval_runtime': 5.3283, 'eval_samples_per_second': 153.521, 'eval_steps_per_second': 19.331, 'epoch': 1.0}
{'loss': 1.5573, 'learning_rate': 3.661957819339435e-05, 'epoch': 1.0}
{'loss': 1.6778, 'learning_rate': 3.652009550338241e-05, 'epoch': 1.01}
{'loss': 1.5912, 'learning_rate': 3.642061281337048e-05, 'epoch': 1.02}
{'loss': 1.5042, 'learning_rate': 3.632113012335854e-05, 'epoch': 1.02}
{'loss': 1.5853, 'learning_rate': 3.6221647433346604e-05, 'epoch': 1.03}
{'loss': 1.6217, 'learning_rate': 3.612216474333466e-05, 'epoch': 1.03}
{'loss': 1.6053, 'learning_rate': 3.6022682053322724e-05, 'epoch': 1.04}
{'loss': 1.4938, 'learning_rate': 3.592319936331078e-05, 'epoch': 1.04}
{'loss': 1.6211, 'learning_rate': 3.5823716673298844e-05, 'epoch': 1.05}
{'loss': 1.5575, 'learning_rate': 3.572423398328691e-05, 'epoch': 1.05}
{'loss': 1.4417, 'learning_rate': 3.562475129327497e-05, 'epoch': 1.06}
{'loss': 1.627, 'learning_rate': 3.5525268603263034e-05, 'epoch':

  0%|          | 0/103 [00:00<?, ?it/s]

{'eval_loss': 1.5237668752670288, 'eval_runtime': 5.2887, 'eval_samples_per_second': 154.669, 'eval_steps_per_second': 19.475, 'epoch': 2.0}
{'loss': 1.4001, 'learning_rate': 1.8324711500198967e-05, 'epoch': 2.0}
{'loss': 1.2514, 'learning_rate': 1.8225228810187027e-05, 'epoch': 2.01}
{'loss': 1.3985, 'learning_rate': 1.812574612017509e-05, 'epoch': 2.01}
{'loss': 1.3864, 'learning_rate': 1.802626343016315e-05, 'epoch': 2.02}
{'loss': 1.2959, 'learning_rate': 1.7926780740151214e-05, 'epoch': 2.02}
{'loss': 1.2653, 'learning_rate': 1.7827298050139278e-05, 'epoch': 2.03}
{'loss': 1.4021, 'learning_rate': 1.772781536012734e-05, 'epoch': 2.04}
{'loss': 1.2711, 'learning_rate': 1.76283326701154e-05, 'epoch': 2.04}
{'loss': 1.2716, 'learning_rate': 1.752884998010346e-05, 'epoch': 2.05}
{'loss': 1.3304, 'learning_rate': 1.7429367290091525e-05, 'epoch': 2.05}
{'loss': 1.2896, 'learning_rate': 1.7329884600079588e-05, 'epoch': 2.06}
{'loss': 1.3458, 'learning_rate': 1.7230401910067648e-05, 'epoc

  0%|          | 0/103 [00:00<?, ?it/s]

{'eval_loss': 1.5043226480484009, 'eval_runtime': 4.9871, 'eval_samples_per_second': 164.025, 'eval_steps_per_second': 20.653, 'epoch': 3.0}
{'train_runtime': 950.3244, 'train_samples_per_second': 46.506, 'train_steps_per_second': 5.815, 'train_loss': 1.6018858128512117, 'epoch': 3.0}


TrainOutput(global_step=5526, training_loss=1.6018858128512117, metrics={'train_runtime': 950.3244, 'train_samples_per_second': 46.506, 'train_steps_per_second': 5.815, 'train_loss': 1.6018858128512117, 'epoch': 3.0})

In [18]:
# Save the model

model.save_pretrained("../models/samsum_bart_base")

In [ ]:
# clear the data left on the gpu
model = None